In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 53.3 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """Constant oracle: f(x)=0 or f(x)=1"""
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn"""
    for q in inputs:
        qc.cx(q, ancilla)




In [4]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    n: number of input qubits
    oracle_func: oracle function to modify the circuit
    oracle_args: extra arguments for oracle
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Step 1: Initialize |0...0>|1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Oracle
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc


In [5]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """Run Deutsch–Jozsa circuit on AerSimulator"""
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    if counts.get("0" * n, 0) == 1024:
        print("✅ Function is CONSTANT")
    else:
        print("✅ Function is BALANCED")


In [6]:
# ---------- MAIN ----------
if __name__ == "__main__":
    n = 3  # number of input qubits

    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0
    )
    print(qc_const.draw(fold=-1))
    run_dj(qc_const)

    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n
    )
    print(qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)



=== Constant Oracle (f(x)=0) ===
     ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
✅ Function is CONSTANT

=== Balanced Oracle (Parity) ===
     ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'111': 1024}
✅ Function is BALANCED


In [7]:
!pip install qiskit qiskit-aer matplotlib qiskit-ibm-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.9 MB/s eta 0:00:00


In [13]:
# Tasks 1-3: Deutsch-Jozsa implementations (constant, balanced parity, balanced-half by MSB),
# runs for different n, and noisy simulation. Visualizes circuits and histograms.
# Requires: qiskit, qiskit-aer, matplotlib

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
import numpy as np

# ---------- ORACLES ----------
def oracle_constant_zero(qc, inputs, ancilla):
    """Constant oracle f(x)=0: do nothing"""
    return

def oracle_constant_one(qc, inputs, ancilla):
    """Constant oracle f(x)=1: flip ancilla always"""
    qc.x(ancilla)

def oracle_balanced_parity(qc, inputs, ancilla):
    """Balanced parity oracle: ancilla flips if parity of inputs is 1"""
    for q in inputs:
        qc.cx(q, ancilla)

def oracle_balanced_msb(qc, inputs, ancilla):
    """
    Balanced oracle that flips ancilla for exactly half of inputs:
    f(x) = value of MSB (most significant input qubit).
    This flips ancilla when MSB == 1 (exactly half of the input states).
    """
    msb = inputs[-1]  # choose last index as MSB
    qc.cx(msb, ancilla)

# ---------- DJ CIRCUIT BUILDER ----------
def deutsch_jozsa_circuit(n, oracle_func, oracle_args=()):
    """
    Build Deutsch-Jozsa circuit with n input qubits + 1 ancilla.
    oracle_func will receive (qc, inputs, ancilla, *oracle_args)
    """
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # initialize ancilla to |1>
    qc.x(ancilla)
    # apply H to all qubits (input + ancilla)
    qc.h(list(range(n + 1)))

    # apply oracle
    if oracle_args:
        oracle_func(qc, inputs, ancilla, *oracle_args)
    else:
        oracle_func(qc, inputs, ancilla)

    # apply H to input qubits
    qc.h(inputs)

    # measure input qubits
    qc.measure(inputs, creg)
    return qc

# ---------- RUNNER ----------
def run_and_plot(qc, shots=1024, noise_model=None, title_suffix=""):
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc, shots=shots, noise_model=noise_model).result()
    counts = result.get_counts()
    print("Counts:", counts)
    plot_histogram(counts, title="Measurement histogram " + title_suffix)
    plt.show()

    n = qc.num_clbits
    zero_state = "0" * n
    if counts.get(zero_state, 0) == shots:
        print("=> Measured all zeros => Function is CONSTANT (expected).")
    else:
        print("=> Non-zero measurement => Function is BALANCED (expected).")

# ---------- NOISE MODEL for Task 3 ----------
def simple_depolarizing_noise_model(p=0.01):
    nm = NoiseModel()
    # depolarizing on single and two-qubit gates
    err1 = depolarizing_error(p, 1)
    err2 = depolarizing_error(p * 2, 2)

    # ✅ Correct: 1q errors → 1q gates, 2q errors → 2q gates
    nm.add_all_qubit_quantum_error(err1, ['x', 'h'])
    nm.add_all_qubit_quantum_error(err2, ['cx'])
    return nm

# ---------- DEMOS ----------
if __name__ == "__main__":

    # ---------- Task 2: Different n values ----------
    for n in [2, 4, 5]:
        print("\n\n--- n =", n, " | Constant oracle (f(x)=0) ---")
        qc_const = deutsch_jozsa_circuit(n, oracle_constant_zero)
        display(qc_const.draw(fold=-1))
        run_and_plot(qc_const, title_suffix=f"(n={n}, constant)")

        print("\n--- n =", n, " | Balanced (parity) oracle ---")
        qc_parity = deutsch_jozsa_circuit(n, oracle_balanced_parity)
        display(qc_parity.draw(fold=-1))
        run_and_plot(qc_parity, title_suffix=f"(n={n}, parity balanced)")

        print("\n--- n =", n, " | Balanced (MSB) oracle — flips ancilla when MSB==1 ---")
        qc_msb = deutsch_jozsa_circuit(n, oracle_balanced_msb)
        display(qc_msb.draw(fold=-1))
        run_and_plot(qc_msb, title_suffix=f"(n={n}, msb balanced)")

    # ---------- Task 3: Noise simulation ----------
    print("\n\n=== Task 3: Noise simulation example (n=3) ===")
    n = 3
    qc_parity = deutsch_jozsa_circuit(n, oracle_balanced_parity)
    nm = simple_depolarizing_noise_model(p=0.02)
    print("Running with simple depolarizing noise (p=0.02)...")
    run_and_plot(qc_parity, shots=2048, noise_model=nm, title_suffix="(noisy parity, n=3)")

    # Compare clean vs noisy
    simulator = AerSimulator()
    tqc_clean = transpile(qc_parity, simulator)
    clean_counts = simulator.run(tqc_clean, shots=2048).result().get_counts()
    noisy_counts = simulator.run(tqc_clean, shots=2048, noise_model=nm).result().get_counts()

    print("\nClean counts:", clean_counts)
    print("Noisy counts:", noisy_counts)
    plot_histogram([clean_counts, noisy_counts],
                   legend=['clean', 'noisy'],
                   title="Clean vs Noisy (parity, n=3)")
    plt.show()



--- n = 2  | Constant oracle (f(x)=0) ---


┌───┐┌───┐┌─┐   
q_0: ┤ H ├┤ H ├┤M├───
     ├───┤├───┤└╥┘┌─┐
q_1: ┤ H ├┤ H ├─╫─┤M├
     ├───┤├───┤ ║ └╥┘
q_2: ┤ X ├┤ H ├─╫──╫─
     └───┘└───┘ ║  ║ 
c: 2/═══════════╩══╩═
                0  1

Counts: {'00': 1024}
=> Measured all zeros => Function is CONSTANT (expected).

--- n = 2  | Balanced (parity) oracle ---


┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1

Counts: {'11': 1024}
=> Non-zero measurement => Function is BALANCED (expected).

--- n = 2  | Balanced (MSB) oracle — flips ancilla when MSB==1 ---


...

Counts: {'00000': 1024}
=> Measured all zeros => Function is CONSTANT (expected).

--- n = 5  | Balanced (parity) oracle ---


┌───┐          ┌───┐     ┌─┐                           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────────────────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐                   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───────────────────
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘     ┌─┐           
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫──────┤M├───────────
     ├───┤       │    │    │   ║ └───┘ ║ ┌───┐└╥┘     ┌─┐   
q_3: ┤ H ├───────┼────┼────┼───╫───■───╫─┤ H ├─╫──────┤M├───
     ├───┤       │    │    │   ║   │   ║ └───┘ ║ ┌───┐└╥┘┌─┐
q_4: ┤ H ├───────┼────┼────┼───╫───┼───╫───■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ ┌─┴─┐ ║ ┌─┴─┐ ║ └───┘ ║ └╥┘
q_5: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫─┤ X ├─╫─┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║ └───┘ ║ └───┘ ║       ║  ║ 
c: 5/══════════════════════════╩═══════╩═══════╩═══════╩══╩═
                               0       1       2       3  4

Counts: {'11111': 1024}
=> Non-zero measurement => Function is BALANCED (expected).

--- n = 5  | Balanced (MSB) oracle — flips ancilla when MSB==1 ---


┌───┐┌───┐     ┌─┐                 
q_0: ┤ H ├┤ H ├─────┤M├─────────────────
     ├───┤├───┤     └╥┘┌─┐              
q_1: ┤ H ├┤ H ├──────╫─┤M├──────────────
     ├───┤├───┤      ║ └╥┘┌─┐           
q_2: ┤ H ├┤ H ├──────╫──╫─┤M├───────────
     ├───┤├───┤      ║  ║ └╥┘┌─┐        
q_3: ┤ H ├┤ H ├──────╫──╫──╫─┤M├────────
     ├───┤└───┘      ║  ║  ║ └╥┘┌───┐┌─┐
q_4: ┤ H ├───────■───╫──╫──╫──╫─┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐ ║  ║  ║  ║ └───┘└╥┘
q_5: ┤ X ├┤ H ├┤ X ├─╫──╫──╫──╫───────╫─
     └───┘└───┘└───┘ ║  ║  ║  ║       ║ 
c: 5/════════════════╩══╩══╩══╩═══════╩═
                     0  1  2  3       4

Counts: {'10000': 1024}
=> Non-zero measurement => Function is BALANCED (expected).


=== Task 3: Noise simulation example (n=3) ===
Running with simple depolarizing noise (p=0.02)...
Counts: {'100': 2, '001': 46, '010': 4, '000': 19, '110': 47, '101': 48, '011': 73, '111': 1809}
=> Non-zero measurement => Function is BALANCED (expected).

Clean counts: {'111': 2048}
Noisy counts: {'100': 3, '000': 20, '001': 33, '101': 59, '110': 61, '011': 91, '111': 1781}


In [14]:
# Task 4: code to run on IBM Quantum device (wrapped safely)
# Task 5: unitary / gate definition printing for oracle circuits
#
# This block is safe to run even if you don't have IBM credentials; it checks and skips.
# Requires qiskit-ibm-runtime if you want to actually run on IBM hardware.

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
import os

# Reuse oracles from Block A (redefining for standalone)
def oracle_constant_one(qc, inputs, ancilla):
    qc.x(ancilla)

def oracle_balanced_parity(qc, inputs, ancilla):
    for q in inputs:
        qc.cx(q, ancilla)

def oracle_balanced_msb(qc, inputs, ancilla):
    qc.cx(inputs[-1], ancilla)

def deutsch_jozsa_circuit_no_meas(n, oracle_func, oracle_args=()):
    """Build DJ circuit but DO NOT include measurements (useful for unitary extraction)."""
    qreg = QuantumRegister(n + 1, "q")
    qc = QuantumCircuit(qreg)

    inputs = list(range(n))
    ancilla = n
    qc.x(ancilla)
    qc.h(list(range(n + 1)))

    # apply oracle (no measurements)
    oracle_func(qc, inputs, ancilla, *oracle_args) if oracle_args else oracle_func(qc, inputs, ancilla)

    qc.h(inputs)
    return qc

def deutsch_jozsa_circuit_with_meas(n, oracle_func, oracle_args=()):
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n
    qc.x(ancilla)
    qc.h(list(range(n + 1)))
    oracle_func(qc, inputs, ancilla, *oracle_args) if oracle_args else oracle_func(qc, inputs, ancilla)
    qc.h(inputs)
    qc.measure(inputs, creg)
    return qc

# ---------- Task 5: Print unitary / gate definition of oracle ----------
def print_oracle_unitary(oracle_func, n, oracle_args=()):
    # Build oracle-only circuit (no initial H or ancilla prep) to inspect gate definition,
    # or build full oracle region and wrap as Gate to print definition.
    qreg = QuantumRegister(n + 1, "q")
    qc_or = QuantumCircuit(qreg)
    inputs = list(range(n))
    ancilla = n
    # prepare ancilla |1> temporarily then revert - but to inspect oracle itself we skip prep
    # We'll create a small circuit that only contains oracle body (so the gate definition shows the oracle)
    oracle_func(qc_or, inputs, ancilla, *oracle_args) if oracle_args else oracle_func(qc_or, inputs, ancilla)
    try:
        gate = qc_or.to_gate(label="oracle")
        print("Oracle gate definition:")
        print(gate.definition)
        # get matrix
        U = Operator(gate).data
        print("Oracle unitary shape:", U.shape)
    except Exception as e:
        print("Could not convert oracle to gate/unitary (maybe uses measurements or unsupported controls):", e)

# ---------- Task 4: Run on IBM Quantum Device (safe wrapper) ----------
def run_on_ibm(qc, shots=1024):
    """
    Attempts to run the given circuit on IBM Quantum if QiskitRuntimeService token is present
    in the environment variable 'QISKIT_IBM_TOKEN'. If not present, this function will skip.
    """
    try:
        from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator
        token = os.environ.get("QISKIT_IBM_TOKEN", None)
        if token is None:
            print("No IBM token found in environment variable QISKIT_IBM_TOKEN. Skipping IBM execution.")
            print("If you want to run on IBM Quantum, set QISKIT_IBM_TOKEN env var (qiskit_ibm_runtime token).")
            return None
        # instantiate service and run (example with the sampler)
        service = QiskitRuntimeService(token=token)
        backend = service.backends(simulator=False)[0]  # pick the first real backend available
        print("Selected backend:", backend.name)
        # Use sampler via runtime (this is illustrative; adapt as needed)
        with Session(service=service, backend=backend.name) as session:
            sampler = Sampler(session=session)
            # remove classical bits for runtime sampler: sampler takes circuits without classical measures
            qc_no_meas = qc.remove_final_measurements(inplace=False)
            job = sampler.run(qc_no_meas, shots=shots)
            result = job.result()
            counts = result.quasi_dists[0].binary_probabilities()
            print("IBM counts (approx):", counts)
            return counts
    except Exception as e:
        print("IBM runtime execution skipped or failed with exception:", e)
        return None

# ---------- Demo: use n=3 ----------
if __name__ == "__main__":
    n = 3
    print("=== Task 5: Oracle unitary / gate definition (parity oracle, n=3) ===")
    print_oracle_unitary(oracle_balanced_parity, n)

    print("\n=== Build full DJ circuit with measurements and show ===")
    qc = deutsch_jozsa_circuit_with_meas(n, oracle_balanced_msb)
    display(qc.draw(fold=-1))

    print("\n=== Run locally on AerSimulator (clean) ===")
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=1024).result()
    counts = result.get_counts()
    print("Local counts:", counts)
    plot_histogram(counts, title="Local (MSB balanced)"); plt.show()

    print("\n=== Attempt Task 4: Run on IBM backend (if token provided) ===")
    # WARNING: this will only run if you set environment var QISKIT_IBM_TOKEN with a valid token.
    run_on_ibm(qc, shots=1024)

    print("\n=== End of Tasks 1-5 demo ===")

=== Task 5: Oracle unitary / gate definition (parity oracle, n=3) ===
Oracle gate definition:
                    
q_0: ──■────────────
       │            
q_1: ──┼────■───────
       │    │       
q_2: ──┼────┼────■──
     ┌─┴─┐┌─┴─┐┌─┴─┐
q_3: ┤ X ├┤ X ├┤ X ├
     └───┘└───┘└───┘
Oracle unitary shape: (16, 16)

=== Build full DJ circuit with measurements and show ===


┌───┐┌───┐     ┌─┐           
q_0: ┤ H ├┤ H ├─────┤M├───────────
     ├───┤├───┤     └╥┘┌─┐        
q_1: ┤ H ├┤ H ├──────╫─┤M├────────
     ├───┤└───┘      ║ └╥┘┌───┐┌─┐
q_2: ┤ H ├───────■───╫──╫─┤ H ├┤M├
     ├───┤┌───┐┌─┴─┐ ║  ║ └───┘└╥┘
q_3: ┤ X ├┤ H ├┤ X ├─╫──╫───────╫─
     └───┘└───┘└───┘ ║  ║       ║ 
c: 3/════════════════╩══╩═══════╩═
                     0  1       2


=== Run locally on AerSimulator (clean) ===
Local counts: {'100': 1024}

=== Attempt Task 4: Run on IBM backend (if token provided) ===
No IBM token found in environment variable QISKIT_IBM_TOKEN. Skipping IBM execution.
If you want to run on IBM Quantum, set QISKIT_IBM_TOKEN env var (qiskit_ibm_runtime token).

=== End of Tasks 1-5 demo ===
